準粒子におけるグリーン関数を用いて、グリーン関数を離散リーマン表現(DLR)や虚数周波数空間内
に変換し、その精度を確認する。
(Sparse-ir tutorial の離散リーマン表現部分の動作確認)

まず、有限温度基底をフェルミオンで作る。

In [ ]:
using SparseIR
using Plots
gr() # USE GR backend
using LaTeXStrings

wmax = 10.0
lambda_ = 1e+4
beta = lambda_/wmax

basis = FiniteTempBasis(Fermionic(), beta, wmax, 1e-15)


print(length(basis))

準粒子のグリーン関数は、


$G^R_{\bm{k}}(\omega)=\frac{1}{\omega-E_{\bm{k}}+i\Gamma_{\bm{k}}}$ 


なので、スペクトル関数は、

$\rho_{\bm{k}}(\omega)=\frac{1}{\pi}\frac{\Gamma_{\bm{k}}}{(\omega-E_{\bm{k}})^2+\Gamma_{\bm{k}}^2}$

但し、$E_{\bm{k}}$は準粒子のエネルギー、$\Gamma_{\bm{k}}$は時定数である。

今、準粒子のエネルギーが定数を取るとすれば、周波数の零点をずらしているだけと捉えられるので、0にしても計算上問題ないとして、0におく。

また、時定数はエネルギーに換算した時、$\beta^{-1}$より十分小さい必要があるので、$\Gamma_{\bm{k}}=10^{-8}$/sとした。

In [ ]:
gamma = 1e-8
E_k = 0.0

rho(omega) = 1/π*gamma/((omega-E_k)^2+gamma^2)


omega = LinRange(-wmax, wmax, 1000)
plot(omega, rho.(omega), xlabel=latexstring("\\omega"), ylabel=latexstring("\\rho(\\omega)"))

有限温度基底とスペクトル関数の重なりを計算

In [ ]:
rhol = overlap(basis.v, rho)
ls = collect(1:length(basis))

plot(ls[1:2:end], abs.(rhol)[1:2:end], marker=:cross, yaxis=:log, ylims=(1,2e1))

グリーン関数を計算

In [ ]:
gl = - basis.s .* rhol

plot(ls[1:2:end], abs.(gl)[1:2:end], marker=:cross, ylims=(1e-20,1e3), yaxis=:log)

グリーン関数をDLRグリーン関数に変換する

In [ ]:
dlr = DiscreteLehmannRepresentation(basis)
g_dlr = SparseIR.from_IR(dlr, gl)

plot(dlr.poles, g_dlr, marker=:cross)

先ほど変換したDLRグリーン関数をグリーン関数に逆変換し、元のグリーン関数からの誤差を見る

誤差は大きいところで$10^{-15}$程度であり、$10^{0}$(その時のグリーン関数のオーダー)に対してとても小さいことが分かる。

In [ ]:
gl_reconst = SparseIR.to_IR(dlr, g_dlr)

plot(
    [abs.(gl), abs.(gl_reconst), abs.(gl-gl_reconst)],
    label=["Exact" "Reconstructed from DLR" "error"],
    marker=[:cross :x :circle], line=(nothing,nothing,nothing), yaxis=:log,
    ylims=(1e-20,100)
)

次は虚数周波数空間でグリーン関数を再構築し、その精度を見る。

In [ ]:
v = FermionicFreq.(2 .* collect(-1000:10:1000) .+ 1)
iv = SparseIR.valueim.(v, beta)

newaxis = [CartesianIndex()]
transmat = 1 ./ (iv[:,newaxis] .- dlr.poles[newaxis,:])
giv = transmat * g_dlr


smpl = MatsubaraSampling(basis; sampling_points=v)
giv_exact = evaluate(smpl, gl)


plot(
    imag.(iv), [imag.(giv_exact), imag.(giv)],label=["Exact" "Reconstruction"], marker=[:cross :x], line=(nothing,nothing,nothing),
    xlabel=latexstring("\\nu"),
    ylabel=latexstring("G(\\mathrm{i}\\omega_n)"),
    )

誤差を表示するとデータが見えにくいため省略したが、誤差は$G=0$の軸上に並んで見えたので、ExactちReconstructionの各点は互いに重なっていることが分かる。

つまり、小さい誤差で変換が可能であり、準粒子のグリーン関数に対してもこれらの変換が有効であるということが確認できた。